In [1]:
import sys
sys.path.append('/workspace/Documents')

import argparse
import os
import sys
import subprocess
import nibabel as nb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import Diffusion_denoising_thin_slice.Build_lists.Build_list as Build_list
import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
build_sheet =  Build_list.Build(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_simulation_train_test_gaussian.xlsx'))
_,patient_id_list,patient_subid_list,random_num_list, condition_list, x0_list = build_sheet.__build__(batch_list = [5]) 
n = ff.get_X_numbers_in_interval(total_number = patient_id_list.shape[0],start_number = 0,end_number = 1, interval = 3)

In [3]:
avg_slice = False

results = []
for i in range(0,1):
    patient_id = patient_id_list[n[i]]
    patient_subid = patient_subid_list[n[i]]
    random_n = random_num_list[n[i]]
    print(patient_id, patient_subid, random_n)

    gt_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_DDPM_gaussian_2D/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch73_1/gt_img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()
    # process gt
    shape = gt_img.shape
    gt_img_new = np.zeros((gt_img.shape[0], gt_img.shape[1], gt_img.shape[2]-2))
    for i in range(1, gt_img.shape[2]-1):
        gt_img_new[:,:,i-1] = np.mean(gt_img[:,:,i-1:i+2], axis = 2)
    gt_img = np.copy(gt_img_new) if avg_slice else np.copy(gt_img)
    gt_img_brain = Data_processing.cutoff_intensity(gt_img, cutoff_low=-100, cutoff_high=100)
    print('done gt')

    condition_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_DDPM_gaussian_2D/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch73_1/condition_img.nii.gz')
    condition_img = nb.load(condition_file).get_fdata() if avg_slice == False else nb.load(condition_file).get_fdata()[:,:,1:shape[2]-1]
    condition_img_brain = Data_processing.cutoff_intensity(condition_img, cutoff_low=-100, cutoff_high=100)
    print('done condition')

    supervised_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_DDPM_possion_2D/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch46_1/pred_img.nii.gz')
    supervised_img = nb.load(supervised_file).get_fdata() if avg_slice == False else nb.load(supervised_file).get_fdata()[:,:,1:shape[2]-1]
    supervised_img_brain = Data_processing.cutoff_intensity(supervised_img, cutoff_low=-100, cutoff_high=100)
    print('done supervised')

    ddpm_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_DDPM_gaussian_2D/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch73_1/pred_img.nii.gz')
    ddpm_img = nb.load(ddpm_file).get_fdata() if avg_slice == False else nb.load(ddpm_file).get_fdata()[:,:,1:shape[2]-1]
    ddpm_img_brain = Data_processing.cutoff_intensity(ddpm_img, cutoff_low=-100, cutoff_high=100)
    print('done ddpm')

    ddpm_avg_20_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_DDPM_gaussian_2D/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch73avg/pred_img_scans20.nii.gz')
    ddpm_avg_20_img = nb.load(ddpm_avg_20_file).get_fdata() if avg_slice == False else nb.load(ddpm_avg_20_file).get_fdata()[:,:,1:shape[2]-1]
    ddpm_avg_20_img_brain = Data_processing.cutoff_intensity(ddpm_avg_20_img, cutoff_low=-100, cutoff_high=100)
    print('done ddpm_avg_20')

    # compare brain region
    mae_brain_motion, _, rmse_brain_motion, _, ssim_brain_motion,psnr_brain_motion = ff.compare(condition_img_brain, gt_img_brain, cutoff_low = 0, cutoff_high = 100)
    mae_brain_supervised, _, rmse_brain_supervised, _, ssim_brain_supervised,psnr_brain_supervised = ff.compare(supervised_img_brain, gt_img_brain, cutoff_low = 0, cutoff_high = 100)

    mae_brain_ddpm, _, rmse_brain_ddpm, _, ssim_brain_ddpm,psnr_brain_ddpm = ff.compare(ddpm_img_brain, gt_img_brain, cutoff_low = 0, cutoff_high = 100)
    mae_brain_ddpm_avg_20, _, rmse_brain_ddpm_avg_20, _, ssim_brain_ddpm_avg_20,psnr_brain_ddpm_avg_20 = ff.compare(ddpm_avg_20_img_brain, gt_img_brain, cutoff_low = 0, cutoff_high = 100)

    print('motion:', mae_brain_motion, rmse_brain_motion, ssim_brain_motion, psnr_brain_motion)
    print('mae_brain_supervised:', mae_brain_supervised, rmse_brain_supervised, ssim_brain_supervised, psnr_brain_supervised)
    print('ddpm:', mae_brain_ddpm, rmse_brain_ddpm, ssim_brain_ddpm, psnr_brain_ddpm)
    print('ddpm_avg_20:', mae_brain_ddpm_avg_20, rmse_brain_ddpm_avg_20, ssim_brain_ddpm_avg_20, psnr_brain_ddpm_avg_20)

00214841 0000455418 0


done gt
done condition
done supervised
done ddpm
done ddpm_avg_20
motion: 7.095717584384185 8.93928718750757 0.838923164987044 20.973942203218634
mae_brain_supervised: 3.035867549330127 4.1442652542776 0.9593984174916785 27.65104910588225
ddpm: 7.413192682170746 9.73376864113613 0.8158671139092611 20.234379611556474
ddpm_avg_20: 3.8497598987977115 5.688263672993708 0.9366092975094237 24.90040561169758


In [4]:
# compare brain region
mae_brain_motion, _, rmse_brain_motion, _, ssim_brain_motion,psnr_brain_motion = ff.compare(condition_img_brain, gt_img_brain, cutoff_low = 10, cutoff_high = 90)
mae_brain_supervised, _, rmse_brain_supervised, _, ssim_brain_supervised,psnr_brain_supervised = ff.compare(supervised_img_brain, gt_img_brain, cutoff_low = 10, cutoff_high = 90)

mae_brain_ddpm, _, rmse_brain_ddpm, _, ssim_brain_ddpm,psnr_brain_ddpm = ff.compare(ddpm_img_brain, gt_img_brain, cutoff_low = 10, cutoff_high = 90)
mae_brain_ddpm_avg_20, _, rmse_brain_ddpm_avg_20, _, ssim_brain_ddpm_avg_20,psnr_brain_ddpm_avg_20 = ff.compare(ddpm_avg_20_img_brain, gt_img_brain, cutoff_low = 10, cutoff_high = 90)

print('motion:', mae_brain_motion, rmse_brain_motion, ssim_brain_motion, psnr_brain_motion)
print('mae_brain_supervised:', mae_brain_supervised, rmse_brain_supervised, ssim_brain_supervised, psnr_brain_supervised)
print('ddpm:', mae_brain_ddpm, rmse_brain_ddpm, ssim_brain_ddpm, psnr_brain_ddpm)
print('ddpm_avg_20:', mae_brain_ddpm_avg_20, rmse_brain_ddpm_avg_20, ssim_brain_ddpm_avg_20, psnr_brain_ddpm_avg_20)

motion: 7.15266540711374 9.002562769140239 0.7851266156468828 19.997527026528857
mae_brain_supervised: 2.9618809629324487 4.02653417059226 0.9462754354364445 26.98622240837507
ddpm: 7.317602603725698 9.546090560159925 0.7609126479915677 19.48833918803286
ddpm_avg_20: 3.6422442590378843 5.30993087223411 0.9223749043767365 24.583072844380062


In [5]:
affine = nb.load(gt_file).affine

gt_img_brain2 = np.copy(gt_img_brain)

gt_img_brain2[gt_img_brain <= 10] = 0
gt_img_brain2[gt_img_brain >= 90] = 0
gt_img_brain2[(gt_img_brain > 10) & (gt_img_brain < 90)] = 1

nb.save(nb.Nifti1Image(gt_img_brain2, affine), os.path.join('/mnt/camca_NAS/denoising/binary.nii.gz'))

In [6]:
affine = nb.load(gt_file).affine

# save gt_img_brain
nb.save(nb.Nifti1Image(gt_img_brain, affine), os.path.join('/mnt/camca_NAS/denoising/gt.nii.gz'))

# save condition_img_brain
nb.save(nb.Nifti1Image(condition_img_brain, affine), os.path.join('/mnt/camca_NAS/denoising/condition.nii.gz'))

# save supervised_img_brain
nb.save(nb.Nifti1Image(supervised_img_brain, affine), os.path.join('/mnt/camca_NAS/denoising/supervised.nii.gz'))
nb.save(nb.Nifti1Image(np.abs(supervised_img_brain-gt_img_brain), affine), os.path.join('/mnt/camca_NAS/denoising/supervised_diff.nii.gz'))

# save ddpm_img_brain
nb.save(nb.Nifti1Image(ddpm_img_brain, affine), os.path.join('/mnt/camca_NAS/denoising/ddpm.nii.gz'))

# save ddpm_avg_20_img_brain
nb.save(nb.Nifti1Image(ddpm_avg_20_img_brain, affine), os.path.join('/mnt/camca_NAS/denoising/ddpm_avg_20.nii.gz'))
nb.save(nb.Nifti1Image(np.abs(ddpm_avg_20_img_brain-gt_img_brain), affine), os.path.join('/mnt/camca_NAS/denoising/ddpm_avg_20_diff.nii.gz'))



In [7]:
nb.save(nb.Nifti1Image(np.abs(condition_img_brain-gt_img_brain), affine), os.path.join('/mnt/camca_NAS/denoising/condition_diff.nii.gz'))

nb.save(nb.Nifti1Image(np.abs(ddpm_img_brain-gt_img_brain), affine), os.path.join('/mnt/camca_NAS/denoising/ddpm_diff.nii.gz'))